In [22]:
#load the libraries

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load
import pickle

In [33]:
data = pd.read_csv('datasets_228_482_diabetes.csv')

In [11]:
data.Outcome.value_counts()

0    499
1    268
Name: Outcome, dtype: int64

In [13]:
from pandas_profiling import ProfileReport
eda_profiling = ProfileReport(data)

ModuleNotFoundError: No module named 'pandas_profiling'

In [14]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [15]:
data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [16]:
def impute_median(data, var):   
    temp = data[data[var].notnull()]
    temp = temp[[var, 'Outcome']].groupby(['Outcome'])[[var]].median()
    data.loc[(data['Outcome'] == 0 ) & (data[var].isnull()), var] = temp.loc[0 ,var]
    data.loc[(data['Outcome'] == 1 ) & (data[var].isnull()), var] = temp.loc[1 ,var]
    return data

In [17]:
data = impute_median(data, 'Glucose')
data = impute_median(data, 'BloodPressure')
data = impute_median(data, 'SkinThickness')
data = impute_median(data, 'Insulin')
data = impute_median(data, 'BMI')

In [18]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [19]:
y = data['Outcome']
x = data.drop('Outcome', axis = 1)
columns = x.columns

scaler = StandardScaler()
scaler = scaler.fit(x)
X = scaler.transform(x)
features = pd.DataFrame(X, columns = columns)


In [26]:
dump(scaler, '../models/scaler.joblib')

TypeError: file must have a 'write' attribute

In [27]:
features.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.638727,0.863334,-0.032418,0.665048,0.310866,0.168996,0.467638,1.424637
1,-0.845829,-1.205867,-0.528051,-0.009825,-0.441186,-0.848430,-0.365494,-0.191778
2,1.232549,2.012890,-0.693262,0.327611,0.310866,-1.328074,0.603475,-0.106704
3,-0.845829,-1.074489,-0.528051,-0.684699,-0.536596,-0.630410,-0.920915,-1.042523
4,-1.142741,0.502045,-2.675794,0.665048,0.294029,1.549789,5.481522,-0.021629


In [28]:
x_train, x_test, y_train, y_test = train_test_split(features, y, test_size = 0.2, random_state = 42)


In [29]:
model = RandomForestClassifier(n_estimators=300, bootstrap = True, max_features = 'sqrt')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [30]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.91      0.91      0.91       103
           1       0.82      0.82      0.82        51

    accuracy                           0.88       154
   macro avg       0.87      0.87      0.87       154
weighted avg       0.88      0.88      0.88       154



In [31]:
dump(model, '../models/model.joblib')


FileNotFoundError: [Errno 2] No such file or directory: '../models/model.joblib'